# Protein Ligand Complex MD Setup tutorial using BioExcel Building Blocks (biobb)

## Introduction

### Based on the official Gromacs tutorial: http://www.mdtutorials.com/gmx/complex/index.html
***
This tutorial aims to illustrate the process of **setting up a simulation system** containing a **protein in complex with a ligand**, step by step, using the **BioExcel Building Blocks library (biobb)**. The particular example used is the **T4 lysozyme** L99A/M102Q protein (PDB code 3HTB), in complex with the **2-propylphenol** small molecule (3-letter Code JZ4). 
***
**Biobb modules** used:

 - [biobb_io](https://github.com/bioexcel/biobb_io): Tools to fetch biomolecular data from public databases.
 - [biobb_model](https://github.com/bioexcel/biobb_model): Tools to model macromolecular structures.
 - [biobb_chemistry](https://github.com/bioexcel/biobb_chemistry): Tools to manipulate chemical data.
 - [biobb_md](https://github.com/bioexcel/biobb_md): Tools to setup and run Molecular Dynamics simulations.
 - [biobb_analysis](https://github.com/bioexcel/biobb_analysis): Tools to analyse Molecular Dynamics trajectories.
 - [biobb_structure_utils](https://github.com/bioexcel/biobb_structure_utils):  Tools to modify or extract information from a PDB structure file.
 
**Auxiliar libraries** used:

 - [nb_conda_kernels](https://github.com/Anaconda-Platform/nb_conda_kernels): Enables a Jupyter Notebook or JupyterLab application in one conda environment to access kernels for Python, R, and other languages found in other environments.
 - [nglview](http://nglviewer.org/#nglview): Jupyter/IPython widget to interactively view molecular structures and trajectories in notebooks.
 - [ipywidgets](https://github.com/jupyter-widgets/ipywidgets): Interactive HTML widgets for Jupyter notebooks and the IPython kernel.
 - [os](https://docs.python.org/3/library/os.html): Python miscellaneous operating system interfaces
 - [plotly](https://plot.ly/python/offline/): Python interactive graphing library integrated in Jupyter notebooks.
 - [simpletraj](https://github.com/arose/simpletraj): Lightweight coordinate-only trajectory reader based on code from GROMACS, MDAnalysis and VMD.


### Conda Installation and Launch

```console
git clone https://github.com/bioexcel/biobb_wf_protein-complex_md_setup.git
cd biobb_wf_protein-complex_md_setup
conda env create -f conda_env/environment.yml
conda activate biobb_Protein-Complex_MDsetup_tutorial
jupyter-nbextension enable --py --user widgetsnbextension
jupyter-nbextension enable --py --user nglview
jupyter-notebook biobb_wf_protein-complex_md_setup/notebooks/biobb_Protein-Complex_MDsetup_tutorial.ipynb
  ``` 

***
### Pipeline steps:
 1. [Introduction](#intro)
 2. [Input Parameters](#input)
 3. [Fetching PDB Structure](#fetch)
 4. [Fix Protein Structure](#fix)
 5. [Create Protein System Topology](#top)
 6. [Create ligand system topology](#ligtop)
 7. [Preparing Ligand Restraints](#restraints)
 8. [Create new protein-ligand complex structure file](#complex)
 9. [Create new protein-ligand complex topology file](#complextop)
 10. [Create Solvent Box](#box)
 11. [Fill the Box with Water Molecules](#water)
 12. [Adding Ions](#ions)
 13. [Energetically Minimize the System](#min)
 14. [Equilibrate the System (NVT)](#nvt)
 15. [Equilibrate the System (NPT)](#npt)
 16. [Free Molecular Dynamics Simulation](#free)
 17. [Post-processing and Visualizing Resulting 3D Trajectory](#post)
 18. [Output Files](#output)
 19. [Questions & Comments](#questions)
 
***
***
<img src="https://bioexcel.eu/wp-content/uploads/2019/04/Bioexcell_logo_1080px_transp.png" alt="Bioexcel2 logo"
	title="Bioexcel2 logo" width="400" />
***


<a id="input"></a>
## Input parameters
**Input parameters** needed:
 - **pdbCode**: PDB code of the protein-ligand complex structure (e.g. 3HTB)
 - **ligandCode**: Small molecule 3-letter code for the ligand structure (e.g. JZ4)
 - **mol_charge**: Charge of the small molecule, needed to add hydrogen atoms.

In [1]:
import nglview
import ipywidgets
import os
import zipfile

pdbCode    = "3HTB"
ligandCode = "JZ4"
mol_charge = 0

<a id="fetch"></a>
***
## Fetching PDB structure
Downloading **PDB structure** with the **protein-ligand complex** from the RCSB PDB database.<br>
Alternatively, a PDB file can be used as starting structure. <br>
Splitting the molecule in **three different files**: 
- **proteinFile**: Protein structure
- **ligandFile**: Ligand structure
- **complexFile**: Protein-ligand complex structure 

***
**Building Blocks** used:
 - [Pdb](https://biobb-io.readthedocs.io/en/latest/api.html#module-api.pdb) from **biobb_io.api.pdb**
***

### Downloading PDB file 

In [2]:
# Downloading desired PDB file 
# Import module
from biobb_io.api.pdb import pdb

# Create properties dict and inputs/outputs
downloaded_pdb = pdbCode + '.orig.pdb'
prop = {
    'pdb_code': pdbCode,
    'filter': False
}

# Create and launch bb
pdb (output_pdb_path=downloaded_pdb, properties=prop)

2022-04-20 15:25:54,867 [MainThread  ] [INFO ]  Downloading: 3htb from: https://www.ebi.ac.uk/pdbe/entry-files/download/pdb3htb.ent


/opt/miniconda3/envs/biobb_env/lib/python3.7/site-packages/urllib3/connectionpool.py:1050: InsecureRequestWarning: Unverified HTTPS request is being made to host 'www.ebi.ac.uk'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  InsecureRequestWarning,


2022-04-20 15:25:55,962 [MainThread  ] [INFO ]  Writting pdb to: 3HTB.orig.pdb


0

### Extracting Protein, Ligand and Protein-Ligand Complex

In [3]:
# Extracting Protein, Ligand and Protein-Ligand Complex to three different files
# Import module
from biobb_structure_utils.utils.extract_heteroatoms import extract_heteroatoms
from biobb_structure_utils.utils.extract_molecule import extract_molecule
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Create inputs/outputs
proteinFile = pdbCode     +'.pdb'
ligandFile  = ligandCode  +'.pdb'
complexFile = pdbCode+'_' + ligandCode + '.pdb'
print (">>> Files:", proteinFile, ligandFile, complexFile)

# Extract molecule
print ("\n>>> Extracting molecule...")
extract_molecule(input_structure_path=downloaded_pdb,
     output_molecule_path=proteinFile)

# Extract ligand 
print ("\n>>> Extracting ligand...")
prop = {
     'heteroatoms' : [{"name": "JZ4"}]
}

extract_heteroatoms(input_structure_path=downloaded_pdb,
     output_heteroatom_path=ligandFile, properties=prop)


# Create complex
print ("\n>>> Creating complex...")
cat_pdb(input_structure1=proteinFile, input_structure2=ligandFile,
       output_structure_path=complexFile)

>>> Files: 3HTB.pdb JZ4.pdb 3HTB_JZ4.pdb

>>> Extracting molecule...
2022-04-20 15:26:08,007 [MainThread  ] [INFO ]  Creating 44b9c974-47d0-4b0a-a6fd-96ea32921ebf temporary folder
2022-04-20 15:26:08,572 [MainThread  ] [INFO ]  check_structure -i /home/lg/repos/javerianaMD/simulation/md-biobb/tutorial/notebooks/3HTB.orig.pdb -o 3HTB.pdb --force_save --non_interactive command_list --list 44b9c974-47d0-4b0a-a6fd-96ea32921ebf/extract_prot.lst

2022-04-20 15:26:08,573 [MainThread  ] [INFO ]  Exit code 0

2022-04-20 15:26:08,574 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure /home/lg/repos/javerianaMD/simulation/md-biobb/tutorial/notebooks/3HTB.orig.pdb loaded
 Title: 2-propylphenol in complex with t4 lysozyme l99a/m102q
 Experimental method: x-ray diffraction
 Keywo

0

### Visualizing 3D structures
Visualizing the generated **PDB structures** using **NGL**:  
- **Protein structure** (Left)
- **Ligand structure** (Center)
- **Protein-ligand complex** (Right)  

In [4]:
# Show structures: protein, ligand and protein-ligand complex
view1 = nglview.show_structure_file(proteinFile)
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(ligandFile)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(complexFile)
view3.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<img src='images/ngl1.png' style='float: left;width:33%'></img><img src='images/ngl2.png' style='float: left;width:23%'></img><img src='images/ngl3.png' style='float: left;width:33%'></img>

<a id="fix"></a>
***
## Fix protein structure
**Checking** and **fixing** (if needed) the protein structure:<br>
- **Modeling** **missing side-chain atoms**, modifying incorrect **amide assignments**, choosing **alternative locations**.<br>
- **Checking** for missing **backbone atoms**, **heteroatoms**, **modified residues** and possible **atomic clashes**.
***
**Building Blocks** used:
 - [FixSideChain](https://biobb-model.readthedocs.io/en/latest/model.html#module-model.fix_side_chain) from **biobb_model.model.fix_side_chain**

Class to model the missing atoms in amino acid side chains of a PDB.
Model the missing atoms in amino acid side chains of a PDB using biobb_structure_checking if the use_modeller property is added the Modeller suite will also be used to rebuild the missing atoms.
***

In [5]:
# Check & Fix Protein Structure
# Import module
from biobb_model.model.fix_side_chain import fix_side_chain

# Create prop dict and inputs/outputs
fixed_pdb = pdbCode +  '_fixed.pdb'

# Create and launch bb
fix_side_chain(input_pdb_path=proteinFile, 
               output_pdb_path=fixed_pdb)

2022-04-20 15:26:22,181 [MainThread  ] [INFO ]  check_structure -i 3HTB.pdb -o 3HTB_fixed.pdb --force_save fixside --fix ALL

2022-04-20 15:26:22,183 [MainThread  ] [INFO ]  Exit code 0

2022-04-20 15:26:22,183 [MainThread  ] [INFO ]  ===============================================================================
=                   BioBB structure checking utility v3.8.1                   =
=                 A. Hospital, P. Andrio, J.L. Gelpi 2018-21                  =

Structure 3HTB.pdb loaded
 Title: 
 Experimental method: unknown
 Resolution (A): N.A.

 Num. models: 1
 Num. chains: 1 (A: Protein)
 Num. residues:  163
 Num. residues with ins. codes:  0
 Num. HETATM residues:  0
 Num. ligands or modified residues:  0
 Num. water mol.:  0
 Num. atoms:  1300

Running fixside. Options: --fix ALL
No residues with missing or unknown side chain atoms found
Structure not modified, saving due to --force_save option
Final Num. models: 1
Final Num. chains: 1 (A: Protein)
Final Num. residues: 

0

In [6]:
# Show structures: protein, ligand and protein-ligand complex
view1 = nglview.show_structure_file(fixed_pdb, default_representation=False)
view1._remote_call('setSize', target='Widget', args=['450px','500px'])
view1.camera='orthographic'
view1.add_licorice(
    "sidechainAttached "
)  # color="#FF0000" or color=0xFF0000 would also work!
view1.center()
view1

NGLWidget()

<a id="top"></a>
***
## Create protein system topology
**Building GROMACS topology** corresponding to the protein structure.<br>
- Force field used in this tutorial is [**amber99sb-ildn**](https://dx.doi.org/10.1002%2Fprot.22711): AMBER **parm99** force field with **corrections on backbone** (sb) and **side-chain torsion potentials** (ildn). 
- Water molecules type used in this tutorial is [**spc/e**](https://pubs.acs.org/doi/abs/10.1021/j100308a038).<br>
- Adding **hydrogen atoms** if missing. Automatically identifying **disulfide bridges**. <br>

Generating two output files: 
- **GROMACS structure** (gro file)
- **GROMACS topology** ZIP compressed file containing:
    - *GROMACS topology top file* (top file)
    - *GROMACS position restraint file/s* (itp file/s)
    
***
**Building Blocks** used:
 - [Pdb2gmx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.pdb2gmx) from **biobb_md.gromacs.pdb2gmx**

The GROMACS pdb2gmx module, reads a .pdb (or .gro) file, reads some database files, adds hydrogens to the molecules and generates coordinates in GROMACS (GROMOS), or optionally .pdb, format and a topology in GROMACS format. These files can subsequently be processed to generate a run input file.
***

In [7]:
# Create Protein system topology
# Import module
from biobb_md.gromacs.pdb2gmx import pdb2gmx

# Create inputs/outputs
output_pdb2gmx_gro     = pdbCode + '_pdb2gmx.gro'
output_pdb2gmx_top_zip = pdbCode + '_pdb2gmx_top.zip'
prop = {
    'force_field' : 'amber99sb-ildn',
    'water_type': 'spce'
}

# Create and launch bb
pdb2gmx(input_pdb_path=fixed_pdb,
        output_gro_path=output_pdb2gmx_gro,
        output_top_zip_path=output_pdb2gmx_top_zip,
        properties=prop)

2022-04-20 15:26:26,883 [MainThread  ] [INFO ]  GROMACS Pdb2gmx 20191 version detected
2022-04-20 15:26:26,884 [MainThread  ] [INFO ]  Not using any container
2022-04-20 15:26:27,422 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright pdb2gmx -f 3HTB_fixed.pdb -o 3HTB_pdb2gmx.gro -p p2g.top -water spce -ff amber99sb-ildn -i posre.itp

2022-04-20 15:26:27,423 [MainThread  ] [INFO ]  Exit code 0

2022-04-20 15:26:27,424 [MainThread  ] [INFO ]  
Using the Amber99sb-ildn force field in directory amber99sb-ildn.ff

going to rename amber99sb-ildn.ff/aminoacids.r2b
going to rename amber99sb-ildn.ff/dna.r2b
going to rename amber99sb-ildn.ff/rna.r2b
Reading 3HTB_fixed.pdb...
Read '', 1364 atoms
Analyzing pdb file
Splitting chemical chains based on TER records or chain id changing.
There are 1 chains and 0 blocks of water and 163 residues with 1364 atoms

  chain  #res #atoms
  1 'A'   163   1364  

Reading residue database... (Amber99sb-ildn)
Processing chain 1 'A' (1364 atoms, 163 residues)
Ide

0

<a id="ligtop"></a>
***
## Create ligand system topology
**Building GROMACS topology** corresponding to the ligand structure.<br>
Force field used in this tutorial step is **amberGAFF**: [General AMBER Force Field](http://ambermd.org/antechamber/gaff.html), designed for rational drug design.<br>
- [Step 1](#ligandTopologyStep1): Add **hydrogen atoms** if missing.
- [Step 2](#ligandTopologyStep2): **Energetically minimize the system** with the new hydrogen atoms. 
- [Step 3](#ligandTopologyStep3): Generate **ligand topology** (parameters). 

***
**Building Blocks** used:
 - [ReduceAddHydrogens](https://biobb-chemistry.readthedocs.io/en/latest/ambertools.html#module-ambertools.reduce_add_hydrogens) from **biobb_chemistry.ambertools.reduce_add_hydrogens**: 
     * Module for adding hydrogens to a given structure.
 
 
 - [BabelMinimize](https://biobb-chemistry.readthedocs.io/en/latest/babelm.html#module-babelm.babel_minimize) from **biobb_chemistry.babelm.babel_minimize**:
     * Energetically minimizes small molecules. 
     
 
 - [AcpypeParamsGMX](https://biobb-chemistry.readthedocs.io/en/latest/acpype.html#module-acpype.acpype_params_gmx) from **biobb_chemistry.acpype.acpype_params_gmx**:
     * Acpype is a tool based in Python to use Antechamber to generate topologies for chemical compounds and to interface with others python applications like CCPN or ARIA
***

<a id="ligandTopologyStep1"></a>
### Step 1: Add **hydrogen atoms**

In [8]:
# Create Ligand system topology, STEP 1
# Reduce_add_hydrogens: add Hydrogen atoms to a small molecule (using Reduce tool from Ambertools package)
# Import module
from biobb_chemistry.ambertools.reduce_add_hydrogens import reduce_add_hydrogens

# Create prop dict and inputs/outputs
output_reduce_h = ligandCode + '.reduce.H.pdb' 
prop = {
    'nuclear' : 'true'
}

# Create and launch bb
reduce_add_hydrogens(input_path=ligandFile,
                   output_path=output_reduce_h,
                   properties=prop)


2022-04-20 15:26:43,255 [MainThread  ] [INFO ]  Not using any container
2022-04-20 15:26:43,356 [MainThread  ] [INFO ]  reduce -NUClear -OH -ROTNH3 -ALLALT JZ4.pdb > JZ4.reduce.H.pdb

2022-04-20 15:26:43,358 [MainThread  ] [INFO ]  Exit code 0

2022-04-20 15:26:43,359 [MainThread  ] [INFO ]  reduce: version 3.3 06/02/2016, Copyright 1997-2016, J. Michael Word
Processing file: "JZ4.pdb"
Database of HETATM connections: "/opt/miniconda3/envs/biobb_env//dat/reduce_wwPDB_het_dict.txt"
VDW dot density = 16/A^2
Orientation penalty scale = 1 (100%)
Eliminate contacts within 3 bonds.
Ignore atoms with |occupancy| <= 0.01 during adjustments.
Waters ignored if B-Factor >= 40 or |occupancy| < 0.66
Aromatic rings in amino acids accept hydrogen bonds.
Building or keeping OH & SH Hydrogens.
Rotating NH3 Hydrogens.
Not processing Met methyls.
 Singles(size 1): A 167 JZ4 OAB 
 orientation 1: A 167 JZ4 OAB :   rot  120: bump=0.000, HB=0.000, total=0.000
Found 0 hydrogens (0 hets)
Standardized 0 hydrogen

0

<a id="ligandTopologyStep2"></a>
### Step 2: **Energetically minimize the system** with the new hydrogen atoms. 

In [9]:
# Create Ligand system topology, STEP 2
# Babel_minimize: Structure energy minimization of a small molecule after being modified adding hydrogen atoms
# Import module
from biobb_chemistry.babelm.babel_minimize import babel_minimize

# Create prop dict and inputs/outputs
output_babel_min = ligandCode + '.H.min.mol2'                              
prop = {
    'method' : 'sd',
    'criteria' : '1e-10',
    'force_field' : 'GAFF'
}

# Create and launch bb
babel_minimize(input_path = output_reduce_h,
              output_path = output_babel_min,
              properties  = prop)

2022-04-20 15:26:54,270 [MainThread  ] [INFO ]  Hydrogens  is not correct, assigned default value: False
2022-04-20 15:26:54,272 [MainThread  ] [INFO ]  Steps  is not correct, assigned default value: 2500
2022-04-20 15:26:54,273 [MainThread  ] [INFO ]  Cut-off  is not correct, assigned default value: False
2022-04-20 15:26:54,274 [MainThread  ] [INFO ]  Rvdw  is not correct, assigned default value: 6.0
2022-04-20 15:26:54,276 [MainThread  ] [INFO ]  Rele  is not correct, assigned default value: 10.0
2022-04-20 15:26:54,277 [MainThread  ] [INFO ]  Frequency  is not correct, assigned default value: 10
2022-04-20 15:26:54,278 [MainThread  ] [INFO ]  Not using any container
2022-04-20 15:26:54,758 [MainThread  ] [INFO ]  obminimize -c 1e-10 -sd -ff GAFF -ipdb JZ4.reduce.H.pdb -omol2 > JZ4.H.min.mol2

2022-04-20 15:26:54,759 [MainThread  ] [INFO ]  Exit code 0

2022-04-20 15:26:54,759 [MainThread  ] [INFO ]  
A T O M   T Y P E S

IDX	TYPE	RING
1	c3	NO
2	ca	AR
3	ca	AR
4	ca	AR
5	ca	AR
6	ca	AR

0

### Visualizing 3D structures
Visualizing the small molecule generated **PDB structures** using **NGL**:  
- **Original Ligand Structure** (Left)
- **Ligand Structure with hydrogen atoms added** (with Reduce program) (Center)
- **Ligand Structure with hydrogen atoms added** (with Reduce program), **energy minimized** (with Open Babel) (Right) 

In [10]:
# Show different structures generated (for comparison)

view1 = nglview.show_structure_file(ligandFile)
view1.add_representation(repr_type='ball+stick')
view1._remote_call('setSize', target='Widget', args=['350px','400px'])
view1.camera='orthographic'
view1
view2 = nglview.show_structure_file(output_reduce_h)
view2.add_representation(repr_type='ball+stick')
view2._remote_call('setSize', target='Widget', args=['350px','400px'])
view2.camera='orthographic'
view2
view3 = nglview.show_structure_file(output_babel_min)
view3.add_representation(repr_type='ball+stick')
view3._remote_call('setSize', target='Widget', args=['350px','400px'])
view3.camera='orthographic'
view3
ipywidgets.HBox([view1, view2, view3])

<img src='images/ngl4-ngl5-ngl6.png' style='float: left;width:70%'></img>

<a id="ligandTopologyStep3"></a>
### Step 3: Generate **ligand topology** (parameters).
We use here ACPYPE tool (http://www.biomedcentral.com/1756-0500/5/367), a wrapper script around the ANTECHAMBER software (http://ambermd.org/) to generate topologies for chemical compounds. Topologies files to be generated so far: CNS/XPLOR, GROMACS, CHARMM and AMBER. Topologies generated by acpype/Antechamber are based on General Amber Force Field (GAFF) and should be used only with compatible force fields like AMBER and its variants.

In [11]:
# Create Ligand system topology, STEP 3
# Acpype_params_gmx: Generation of topologies for GROMACS with ACPype
# Import module
from biobb_chemistry.acpype.acpype_params_gmx import acpype_params_gmx

# Create prop dict and inputs/outputs
output_acpype_gro = ligandCode + '_params.gro'
output_acpype_itp = ligandCode + '_params.itp'
output_acpype_top = ligandCode + '_params.top'
output_acpype     = ligandCode + '_params'
prop = {
    'basename' : output_acpype,
    'charge' : mol_charge
}

# Create and launch bb
acpype_params_gmx(input_path=output_babel_min, 
                output_path_gro=output_acpype_gro,
                output_path_itp=output_acpype_itp,
                output_path_top=output_acpype_top,
                properties=prop)

2022-04-20 15:28:03,882 [MainThread  ] [INFO ]  Running acpype, this execution can take a while
2022-04-20 15:28:03,883 [MainThread  ] [INFO ]  Not using any container
2022-04-20 15:28:05,600 [MainThread  ] [INFO ]  acpype -i /home/lg/repos/javerianaMD/simulation/md-biobb/tutorial/notebooks/JZ4.H.min.mol2 -b JZ4_params.IOtwE2 -n 0

2022-04-20 15:28:05,602 [MainThread  ] [INFO ]  Exit code 0

2022-04-20 15:28:05,602 [MainThread  ] [INFO ]  ========================================================================================
| ACPYPE: AnteChamber PYthon Parser interfacE v. 2019-11-07T23:16:00CET (c) 2022 AWSdS |
==> ... charge set to 0
==> Executing Antechamber...
==> * Antechamber OK *
==> * Parmchk OK *
==> Executing Tleap...
++++++++++start_quote+++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
Checking 'JZ4'....
Checking parameters for unit 'JZ4'.
Checking for bond parameters.
Checking for angle parameters.
Unit is OK.
++++++++++end_quote+++++++++++++++++++++++++++++++++

0

<a id="restraints"></a>
***
## Preparing Ligand Restraints
- In subsequent steps of the pipeline, such as the equilibration stages of the **protein-ligand complex** system, it is recommended to apply some **restraints** to the small molecule, to avoid a possible change in position due to protein repulsion. 

- **Position restraints** will be applied to the ligand, using a **force constant of 1000 KJ/mol\*nm^2** on the three coordinates: x, y and z. In this steps the **restriction files** will be created and integrated in the **ligand topology**.
- [Step 1](#restraintsStep1): Creating an index file with a new group including just the **small molecule heavy atoms**.
- [Step 2](#restraintsStep2): Generating the **position restraints** file.
***
**Building Blocks** used:
 - [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_md.gromacs.make_ndx**:
     * The GROMACS make_ndx module, generates an index file using the atoms of the selection.
     
     
 - [Genrestr](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genrestr) from **biobb_md.gromacs.genrestr**:
     * Wrapper of the GROMACS genrestr module. The GROMACS genrestr module, produces an #include file for a topology containing a list of atom numbers and three force constants for the x-, y-, and z-direction based on the contents of the -f file. A single isotropic force constant may be given on the command line instead of three components.
***

<a id="restraintsStep1"></a>
### Step 1: Creating an index file for the small molecule heavy atoms

In [85]:
# MakeNdx: Creating index file with a new group (small molecule heavy atoms)
from biobb_md.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_ligand_ndx = ligandCode + '_index.ndx'
prop = {
    'selection': "0 & ! a H*"
}

# Create and launch bb
make_ndx(input_structure_path=output_acpype_gro,
        output_ndx_path=output_ligand_ndx,
        properties=prop)

2022-04-18 16:50:41,959 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2022-04-18 16:50:41,960 [MainThread  ] [INFO ]  Not using any container
2022-04-18 16:50:41,985 [MainThread  ] [INFO ]  echo -e '0 & ! a H*\nq' | gmx -nobackup -nocopyright make_ndx -f JZ4_params.gro -o JZ4_index.ndx

2022-04-18 16:50:41,986 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 16:50:41,987 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:     1      Other residues
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              :    22 atoms
  1 Other               :    22 atoms
  2 JZ4                 :    22 atoms

 nr : group      '!': not  'name' nr name   'splitch' nr    Enter: list groups
 'a': atom       '&': and  'del' nr         'splitres' nr   'l': list residues
 't': atom type  '|': or   'keep' nr        'splitat' nr    'h': help
 'r': residue              'res' nr         'chain' char
 

0

<a id="restraintsStep2"></a>
### Step 2: Generating the position restraints file

In [86]:
# Genrestr: Generating the position restraints file
from biobb_md.gromacs.genrestr import genrestr

# Create prop dict and inputs/outputs
output_restraints_top = ligandCode + '_posres.itp'
prop = {
    'force_constants': "1000 1000 1000",
    'restrained_group': "System"
}

# Create and launch bb
genrestr(input_structure_path=output_acpype_gro,
         input_ndx_path=output_ligand_ndx,
         output_itp_path=output_restraints_top,
         properties=prop)

2022-04-18 16:51:00,078 [MainThread  ] [INFO ]  GROMACS Genrestr 20191 version detected
2022-04-18 16:51:00,079 [MainThread  ] [INFO ]  Not using any container
2022-04-18 16:51:00,099 [MainThread  ] [INFO ]  echo "System" | gmx -nobackup -nocopyright genrestr -f JZ4_params.gro -o JZ4_posres.itp -fc 1000 1000 1000 -n JZ4_index.ndx

2022-04-18 16:51:00,101 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 16:51:00,101 [MainThread  ] [INFO ]  Select group to position restrain
Selected 0: 'System'

2022-04-18 16:51:00,103 [MainThread  ] [INFO ]                       :-) GROMACS - gmx genrestr, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/biobb_md/notebooks
Command line:
  gmx -nobackup -nocopyright genrestr -f JZ4_params.gro -o JZ4_posres.itp -fc 1000 1000 1000 -n JZ4_index.ndx


Reading structure file
Group     0 (         System) has    22 elements
Group     1 (          Other) has    

0

<a id="complex"></a>
***
## Create new protein-ligand complex structure file
Building new **protein-ligand complex** PDB file with:
- The new **protein system** with fixed problems from *Fix Protein Structure* step and hydrogens atoms added from *Create Protein System Topology* step.
- The new **ligand system** with hydrogens atoms added from *Create Ligand System Topology* step. 

This new structure is needed for **GROMACS** as it is **force field-compliant**, it **has all the new hydrogen atoms**, and the **atom names are matching the newly generated protein and ligand topologies**.
***
**Building Blocks** used:
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str**:
     * Wrapper of the GROMACS trjconv module for converting between GROMACS compatible structure file formats and/or extracting a selection of atoms. GROMACS trjconv module can convert trajectory files in many ways
***

In [87]:
print (pdbCode)
print (ligandCode)
print (output_pdb2gmx_gro)
print (output_acpype_gro)
print (complexFile_H)

3HTB
JZ4
3HTB_pdb2gmx.gro
JZ4_params.gro
3HTB_JZ4_H.pdb


In [89]:
# biobb analysis module
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str
from biobb_structure_utils.utils.cat_pdb import cat_pdb

# Convert gro (with hydrogens) to pdb (PROTEIN)
proteinFile_H = pdbCode + '_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_pdb2gmx_gro,
              input_top_path=output_pdb2gmx_gro,
              output_str_path=proteinFile_H, 
              properties=prop)

# Convert gro (with hydrogens) to pdb (LIGAND)
ligandFile_H = ligandCode + '_complex_H.pdb'
prop = {
    'selection' : 'System'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path=output_acpype_gro,
              input_top_path=output_acpype_gro,
              output_str_path=ligandFile_H, 
              properties=prop)


# Concatenating both PDB files: Protein + Ligand
complexFile_H = pdbCode + '_' + ligandCode + '_complex_H.pdb'

# Create and launch bb
cat_pdb(input_structure1=proteinFile_H,
        input_structure2=ligandFile_H,
        output_structure_path=complexFile_H)

2022-04-18 16:53:46,705 [MainThread  ] [INFO ]  Not using any container
2022-04-18 16:53:46,745 [MainThread  ] [INFO ]  echo "System" | gmx trjconv -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_pdb2gmx.gro -s /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_pdb2gmx.gro -o 3HTB_complex_H.pdb

2022-04-18 16:53:46,746 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 16:53:46,747 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 0: 'System'

2022-04-18 16:53:46,748 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen

0

<a id="complextop"></a>
***
## Create new protein-ligand complex topology file
Building new **protein-ligand complex** GROMACS topology file with:
- The new **protein system** topology generated from *Create Protein System Topology* step.
- The new **ligand system** topology generated from *Create Ligand System Topology* step. 

NOTE: From this point on, the **protein-ligand complex structure and topology** generated can be used in a regular MD setup.
***
**Building Blocks** used:
 - [AppendLigand](https://biobb-md.readthedocs.io/en/latest/modules.html) from **biobb_md.gromacs_extra.append_ligand**  (NOTE: link should be updated with the documentation):
     * This class takes a ligand ITP file and inserts it in a topology.
This module automatizes the process of inserting a ligand ITP file in a GROMACS topology.
***

In [90]:
# AppendLigand: Append a ligand to a GROMACS topology
# Import module
from biobb_md.gromacs_extra.append_ligand import append_ligand

# Create prop dict and inputs/outputs
output_complex_top = pdbCode + '_' + ligandCode + '_complex.top.zip'

posresifdef = "POSRES_"+ligandCode.upper()
prop = {
    'posres_name': posresifdef
}

# Create and launch bb
append_ligand(input_top_zip_path=output_pdb2gmx_top_zip,
             input_posres_itp_path=output_restraints_top,
             input_itp_path=output_acpype_itp, 
             output_top_zip_path=output_complex_top,
             properties=prop)

2022-04-18 17:07:24,718 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_pdb2gmx_top.zip
2022-04-18 17:07:24,719 [MainThread  ] [INFO ]  to:
2022-04-18 17:07:24,720 [MainThread  ] [INFO ]  ['f3db7596-a730-426d-8c73-c1998e993eb0/p2g.top', 'f3db7596-a730-426d-8c73-c1998e993eb0/posre.itp']
2022-04-18 17:07:24,721 [MainThread  ] [INFO ]  Unzipping: 
2022-04-18 17:07:24,722 [MainThread  ] [INFO ]  3HTB_pdb2gmx_top.zip
2022-04-18 17:07:24,723 [MainThread  ] [INFO ]  To: 
2022-04-18 17:07:24,725 [MainThread  ] [INFO ]  f3db7596-a730-426d-8c73-c1998e993eb0/p2g.top
2022-04-18 17:07:24,726 [MainThread  ] [INFO ]  f3db7596-a730-426d-8c73-c1998e993eb0/posre.itp
2022-04-18 17:07:24,758 [MainThread  ] [INFO ]  Compressing topology to: 3HTB_JZ4_complex.top.zip
2022-04-18 17:07:24,759 [MainThread  ] [INFO ]  Ignored file f3db7596-a730-426d-8c73-c1998e993eb0/amber99sb-ildn.ff/forcefield.itp
2022-04-18 17:07:24,772 [MainThread  ] [INFO ]  Ignored file f3db7596-a730-

0

<a id="box"></a>
***
## Create solvent box
Define the unit cell for the **protein-ligand complex** to fill it with water molecules.<br>
**Truncated octahedron** box is used for the unit cell. This box type is the one which best reflects the geometry of the solute/protein, in this case a **globular protein**, as it approximates a sphere. It is also convenient for the computational point of view, as it accumulates **less water molecules at the corners**, reducing the final number of water molecules in the system and making the simulation run faster.<br> A **protein to box** distance of **0.8 nm** is used, and the protein is **centered in the box**.  

***
**Building Blocks** used:
 - [Editconf](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.editconf) from **biobb_md.gromacs.editconf**:
     * The GROMACS solvate module generates a box around the selected structure.
***

In [91]:
# Editconf: Create solvent box
# Import module
from biobb_md.gromacs.editconf import editconf

# Create prop dict and inputs/outputs
output_editconf_gro = pdbCode + '_'+ ligandCode + '_complex_editconf.gro'

prop = {
    'box_type': 'octahedron',
    'distance_to_molecule': 0.8
}

# Create and launch bb
editconf(input_gro_path=complexFile_H, 
         output_gro_path=output_editconf_gro,
         properties=prop)

2022-04-18 17:18:48,544 [MainThread  ] [INFO ]  GROMACS Editconf 20191 version detected
2022-04-18 17:18:48,546 [MainThread  ] [INFO ]  Centering molecule in the box.
2022-04-18 17:18:48,549 [MainThread  ] [INFO ]  Distance of the box to molecule:   0.80
2022-04-18 17:18:48,550 [MainThread  ] [INFO ]  Box type: octahedron
2022-04-18 17:18:48,551 [MainThread  ] [INFO ]  Not using any container
2022-04-18 17:18:48,607 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright editconf -f 3HTB_JZ4_complex_H.pdb -o 3HTB_JZ4_complex_editconf.gro -d 0.8 -bt octahedron -c

2022-04-18 17:18:48,609 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 17:18:48,609 [MainThread  ] [INFO ]  Note that major changes are planned in future for editconf, to improve usability and utility.Read 2636 atoms
Volume: 1322.64 nm^3, corresponds to roughly 595100 electrons
No velocities found
    system size :  4.069  4.192  5.073 (nm)
    diameter    :  5.849               (nm)
    center      :  2.241 -1.669 -0.935 (nm)
   

0

<a id="water"></a>
***
## Fill the box with water molecules
Fill the unit cell for the **protein-ligand complex** with water molecules.<br>
The solvent type used is the default **Simple Point Charge water (SPC)**, a generic equilibrated 3-point solvent model. 

***
**Building Blocks** used:
 - [Solvate](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.solvate) from **biobb_md.gromacs.solvate**:
     * The GROMACS solvate module, generates a box of solvent around the selected structure.
***

In [93]:
# Solvate: Fill the box with water molecules
from biobb_md.gromacs.solvate import solvate

# Create prop dict and inputs/outputs
output_solvate_gro     = pdbCode + '_' + ligandCode + '_solvate.gro'
output_solvate_top_zip = pdbCode + '_' + ligandCode + '_solvate_top.zip'

# Create and launch bb
solvate(input_solute_gro_path=output_editconf_gro,
        output_gro_path=output_solvate_gro,
        input_top_zip_path=output_complex_top,
        output_top_zip_path=output_solvate_top_zip)

2022-04-18 21:11:22,736 [MainThread  ] [INFO ]  GROMACS Solvate 20191 version detected
2022-04-18 21:11:22,742 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_complex.top.zip
2022-04-18 21:11:22,743 [MainThread  ] [INFO ]  to:
2022-04-18 21:11:22,746 [MainThread  ] [INFO ]  ['42bca799-53f7-416a-89e7-fdaf24dd3b65/JZ4_params.itp', '42bca799-53f7-416a-89e7-fdaf24dd3b65/JZ4_posres.itp', '42bca799-53f7-416a-89e7-fdaf24dd3b65/ligand.top', '42bca799-53f7-416a-89e7-fdaf24dd3b65/posre.itp']
2022-04-18 21:11:22,748 [MainThread  ] [INFO ]  Unzipping: 
2022-04-18 21:11:22,750 [MainThread  ] [INFO ]  3HTB_JZ4_complex.top.zip
2022-04-18 21:11:22,751 [MainThread  ] [INFO ]  To: 
2022-04-18 21:11:22,752 [MainThread  ] [INFO ]  42bca799-53f7-416a-89e7-fdaf24dd3b65/JZ4_params.itp
2022-04-18 21:11:22,753 [MainThread  ] [INFO ]  42bca799-53f7-416a-89e7-fdaf24dd3b65/JZ4_posres.itp
2022-04-18 21:11:22,755 [MainThread  ] [INFO ]  42bca799-53f7-416a-89e7-fdaf24dd3b65

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex** with the newly added **solvent box** using **NGL**<br>
Note the **octahedral box** filled with **water molecules** surrounding the **protein structure**, which is **centered** right in the middle of the box.

In [94]:
#Show protein
view = nglview.show_structure_file(output_solvate_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)
view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')
view._remote_call('setSize', target='Widget', args=['','600px'])
view.camera='orthographic'
view

NGLWidget()

<img src='images/ngl7.png' width="50%" height="50%"></img>

<a id="ions"></a>
***
## Adding ions
Add ions to neutralize the **protein-ligand complex** and reach a desired ionic concentration.
- [Step 1](#ionsStep1): Creating portable binary run file for ion generation
- [Step 2](#ionsStep2): Adding ions to **neutralize** the system and reach a **0.05 molar ionic concentration**
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp**:
     * The GROMACS preprocessor module needs to be fed with the input system and the dynamics parameters to create a portable binary run input file TPR.
     
     
 - [Genion](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.genion) from **biobb_md.gromacs.genion**:
     * The GROMACS genion module randomly replaces solvent molecules with monoatomic ions. The group of solvent molecules should be continuous and all molecules should have the same number of atoms.
***

<a id="ionsStep1"></a>
### Step 1: Creating portable binary run file for ion generation

In [95]:
# Grompp: Creating portable binary run file for ion generation
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000'
    },
    'simulation_type':'minimization',
    'maxwarn': 1
}
output_gppion_tpr = pdbCode + '_' + ligandCode + '_complex_gppion.tpr'

# Create and launch bb
grompp(input_gro_path=output_solvate_gro,
       input_top_zip_path=output_solvate_top_zip, 
       output_tpr_path=output_gppion_tpr,
       properties=prop)

2022-04-18 21:14:55,678 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-18 21:14:55,685 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_solvate_top.zip
2022-04-18 21:14:55,686 [MainThread  ] [INFO ]  to:
2022-04-18 21:14:55,687 [MainThread  ] [INFO ]  ['f611baa5-511e-4fa6-a452-c30c626df993/JZ4_params.itp', 'f611baa5-511e-4fa6-a452-c30c626df993/JZ4_posres.itp', 'f611baa5-511e-4fa6-a452-c30c626df993/ligand.top', 'f611baa5-511e-4fa6-a452-c30c626df993/posre.itp']
2022-04-18 21:14:55,692 [MainThread  ] [INFO ]  Unzipping: 
2022-04-18 21:14:55,693 [MainThread  ] [INFO ]  3HTB_JZ4_solvate_top.zip
2022-04-18 21:14:55,694 [MainThread  ] [INFO ]  To: 
2022-04-18 21:14:55,695 [MainThread  ] [INFO ]  f611baa5-511e-4fa6-a452-c30c626df993/JZ4_params.itp
2022-04-18 21:14:55,697 [MainThread  ] [INFO ]  f611baa5-511e-4fa6-a452-c30c626df993/JZ4_posres.itp
2022-04-18 21:14:55,698 [MainThread  ] [INFO ]  f611baa5-511e-4fa6-a452-c30c626df993/

0

<a id="ionsStep2"></a>
### Step 2: Adding ions to neutralize the system and reach a 0.05 molar concentration
Replace **solvent molecules** with **ions** to **neutralize** the system and reaching a **0.05 molar ionic concentration**

In [104]:
# Genion: Adding ions to reach a 0.05 molar concentration
from biobb_md.gromacs.genion import genion

# Create prop dict and inputs/outputs
prop={
    'neutral':True,
    'concentration':0.05
}
output_genion_gro = pdbCode + '_' + ligandCode+'_genion.gro'
output_genion_top_zip = pdbCode + '_' + ligandCode+'_genion_top.zip'

# Create and launch bb
genion(input_tpr_path=output_gppion_tpr,
       output_gro_path=output_genion_gro, 
       input_top_zip_path=output_solvate_top_zip,
       output_top_zip_path=output_genion_top_zip, 
       properties=prop)

2022-04-18 21:23:11,024 [MainThread  ] [INFO ]  GROMACS Genion 20191 version detected
2022-04-18 21:23:11,031 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_solvate_top.zip
2022-04-18 21:23:11,032 [MainThread  ] [INFO ]  to:
2022-04-18 21:23:11,033 [MainThread  ] [INFO ]  ['ea755a23-b9d5-4311-a09f-bad6252e80cf/JZ4_params.itp', 'ea755a23-b9d5-4311-a09f-bad6252e80cf/JZ4_posres.itp', 'ea755a23-b9d5-4311-a09f-bad6252e80cf/ligand.top', 'ea755a23-b9d5-4311-a09f-bad6252e80cf/posre.itp']
2022-04-18 21:23:11,034 [MainThread  ] [INFO ]  Unzipping: 
2022-04-18 21:23:11,034 [MainThread  ] [INFO ]  3HTB_JZ4_solvate_top.zip
2022-04-18 21:23:11,035 [MainThread  ] [INFO ]  To: 
2022-04-18 21:23:11,036 [MainThread  ] [INFO ]  ea755a23-b9d5-4311-a09f-bad6252e80cf/JZ4_params.itp
2022-04-18 21:23:11,037 [MainThread  ] [INFO ]  ea755a23-b9d5-4311-a09f-bad6252e80cf/JZ4_posres.itp
2022-04-18 21:23:11,038 [MainThread  ] [INFO ]  ea755a23-b9d5-4311-a09f-bad6252e80cf/

0

### Visualizing 3D structure
Visualizing the **protein-ligand complex** with the newly added **ionic concentration** using **NGL**

In [103]:
#Show protein
view = nglview.show_structure_file(output_genion_gro)
view.clear_representations()
view.add_representation(repr_type='cartoon', selection='protein', color='sstruc')
view.add_representation(repr_type='licorice', radius='.5', selection=ligandCode)

view.add_representation(repr_type='line', linewidth='1', selection='SOL', opacity='.3')

view.add_representation(repr_type='ball+stick', selection='NA')
view.add_representation(repr_type='ball+stick', selection='CL')
view._remote_call('setSize', target='Widget', args=['','400px'])
view.camera='orthographic'
view.center()
view

NGLWidget()

<img src='images/ngl8.png' width="50%" height="50%"></img>

<a id="min"></a>
***
## Energetically minimize the system
Energetically minimize the **protein-ligand complex** till reaching a desired potential energy.
- [Step 1](#emStep1): Creating portable binary run file for energy minimization
- [Step 2](#emStep2): Energetically minimize the **protein-ligand complex** till reaching a force of 500 kJ mol-1 nm-1.
- [Step 3](#emStep3): Checking **energy minimization** results. Plotting energy by time during the **minimization** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp**:
     * The GROMACS preprocessor module needs to be fed with the input system and the dynamics parameters to create a portable binary run input file TPR.
     
     
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun**:
     * MDRun is the main computational chemistry engine within GROMACS. It performs Molecular Dynamics simulations, but it can also perform Stochastic Dynamics, Energy Minimization, test particle insertion or (re)calculation of energies.
     
     
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy**:
     * GROMACS energy extracts energy components from an energy file.
***

<a id="emStep1"></a>
### Step 1: Creating portable binary run file for energy minimization
Method used to run the **energy minimization** is a **steepest descent**, with a **maximum force of 500 KJ/mol\*nm^2**, and a minimization **step size of 1fs**. The **maximum number of steps** to perform if the maximum force is not reached is **5,000 steps**. 

In [105]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        'nsteps':'5000',
        'emstep': 0.01,
        'emtol':'500'
    },
    'simulation_type':'minimization'
}
output_gppmin_tpr = pdbCode+'_'+ligandCode+'_gppmin.tpr'

# Create and launch bb
grompp(input_gro_path=output_genion_gro,
       input_top_zip_path=output_genion_top_zip,
       output_tpr_path=output_gppmin_tpr,
       properties=prop)

2022-04-18 21:23:22,005 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-18 21:23:22,013 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_genion_top.zip
2022-04-18 21:23:22,013 [MainThread  ] [INFO ]  to:
2022-04-18 21:23:22,014 [MainThread  ] [INFO ]  ['cee19a68-0f7c-4841-8123-fa04552c9d71/JZ4_params.itp', 'cee19a68-0f7c-4841-8123-fa04552c9d71/JZ4_posres.itp', 'cee19a68-0f7c-4841-8123-fa04552c9d71/ligand.top', 'cee19a68-0f7c-4841-8123-fa04552c9d71/posre.itp']
2022-04-18 21:23:22,015 [MainThread  ] [INFO ]  Unzipping: 
2022-04-18 21:23:22,016 [MainThread  ] [INFO ]  3HTB_JZ4_genion_top.zip
2022-04-18 21:23:22,018 [MainThread  ] [INFO ]  To: 
2022-04-18 21:23:22,019 [MainThread  ] [INFO ]  cee19a68-0f7c-4841-8123-fa04552c9d71/JZ4_params.itp
2022-04-18 21:23:22,020 [MainThread  ] [INFO ]  cee19a68-0f7c-4841-8123-fa04552c9d71/JZ4_posres.itp
2022-04-18 21:23:22,021 [MainThread  ] [INFO ]  cee19a68-0f7c-4841-8123-fa04552c9d71/li

0

<a id="emStep2"></a>
### Step 2: Running Energy Minimization
Running **energy minimization** using the **tpr file** generated in the previous step.

In [107]:
# Mdrun: Running minimization
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_min_trr = pdbCode+'_'+ligandCode+'_min.trr'
output_min_gro = pdbCode+'_'+ligandCode+'_min.gro'
output_min_edr = pdbCode+'_'+ligandCode+'_min.edr'
output_min_log = pdbCode+'_'+ligandCode+'_min.log'

# Create and launch bb
mdrun(input_tpr_path=output_gppmin_tpr,
      output_trr_path=output_min_trr, 
      output_gro_path=output_min_gro,
      output_edr_path=output_min_edr, 
      output_log_path=output_min_log)

2022-04-18 21:24:22,278 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-18 21:24:22,280 [MainThread  ] [INFO ]  Not using any container
2022-04-18 21:26:00,627 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppmin.tpr -o 3HTB_JZ4_min.trr -c 3HTB_JZ4_min.gro -e 3HTB_JZ4_min.edr -g 3HTB_JZ4_min.log

2022-04-18 21:26:00,628 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 21:26:00,629 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/biobb_md/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppmin.tpr -o 3HTB_JZ4_min.trr -c 3HTB_JZ4_min.gro -e 3HTB_JZ4_min.edr -g 3HTB_JZ4_min.log

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than the code in
gmx mdrun was configured to use

0

<a id="emStep3"></a>
### Step 3: Checking Energy Minimization results
Checking **energy minimization** results. Plotting **potential energy** by time during the minimization process. 

In [108]:
# GMXEnergy: Getting system energy by time  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_min_ene_xvg = pdbCode + '_' + ligandCode + '_min_ene.xvg'
prop = {
    'terms':  ["Potential"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_min_edr, 
          output_xvg_path=output_min_ene_xvg, 
          properties=prop)

2022-04-18 21:26:00,641 [MainThread  ] [INFO ]  Not using any container
2022-04-18 21:26:00,662 [MainThread  ] [INFO ]  gmx energy -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_min.edr -o 3HTB_JZ4_min_ene.xvg -xvg none < a0eff958-0796-4b9a-b5c2-67aab17ebb89/instructions.in

2022-04-18 21:26:00,663 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 21:26:00,664 [MainThread  ] [INFO ]  
Statistics over 1806 steps [ 0.0000 through 1805.0000 ps ], 1 data sets
All statistics are over 1429 points (frames)

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Potential                   -530532       8600    22279.7   -56368.5  (kJ/mol)

2022-04-18 21:26:00,665 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar 

0

In [111]:
import plotly
import plotly.graph_objs as go

#Read data from file and filter energy values higher than 1000 Kj/mol^-1
with open(output_min_ene_xvg,'r') as energy_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in energy_file 
            if not line.startswith(("#","@")) 
            if float(line.split()[1]) < 1000 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Energy Minimization",
                        xaxis=dict(title = "Energy Minimization Step"),
                        yaxis=dict(title = "Potential Energy KJ/mol-1")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig1.png' width="50%" height="50%"></img>

<a id="nvt"></a>
***
## Equilibrate the system (NVT)
Equilibrate the **protein-ligand complex** system in NVT ensemble (constant Number of particles, Volume and Temperature). To avoid temperature coupling problems, a new *"system"* group will be created including the **protein** + the **ligand** to be assigned to a single thermostatting group.

- [Step 1](#eqNVTStep1): Creating an index file with a new group including the **protein-ligand complex**.
- [Step 2](#eqNVTStep3): Creating portable binary run file for system equilibration
- [Step 3](#eqNVTStep3): Equilibrate the **protein-ligand complex** with NVT ensemble.
- [Step 4](#eqNVTStep4): Checking **NVT Equilibration** results. Plotting **system temperature** by time during the **NVT equilibration** process. 
***
**Building Blocks** used:
- [MakeNdx](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.make_ndx) from **biobb_md.gromacs.make_ndx**:
    * The GROMACS make_ndx module, generates an index file using the atoms of the selection.
    

- [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp**:
    * The GROMACS preprocessor module needs to be fed with the input system and the dynamics parameters to create a portable binary run input file TPR. 
    
    
- [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun**:
    * MDRun is the main computational chemistry engine within GROMACS. It performs Molecular Dynamics simulations, but it can also perform Stochastic Dynamics, Energy Minimization, test particle insertion or (re)calculation of energies.
    
    
- [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy**:
    * GROMACS energy extracts energy components from an energy file. 
***

<a id="eqNVTStep1"></a>
### Step 1: Creating an index file with a new group including the **protein-ligand complex**

In [112]:
# MakeNdx: Creating index file with a new group (protein-ligand complex)
from biobb_md.gromacs.make_ndx import make_ndx

# Create prop dict and inputs/outputs
output_complex_ndx = pdbCode + '_' + ligandCode + '_index.ndx'
prop = {
    'selection': "\"Protein\"|\"Other\"" 
}

# Create and launch bb
make_ndx(input_structure_path=output_min_gro,
        output_ndx_path=output_complex_ndx,
        properties=prop)

2022-04-18 23:18:40,729 [MainThread  ] [INFO ]  GROMACS MakeNdx 20191 version detected
2022-04-18 23:18:40,730 [MainThread  ] [INFO ]  Not using any container
2022-04-18 23:18:40,869 [MainThread  ] [INFO ]  echo -e '"Protein"|"Other"\nq' | gmx -nobackup -nocopyright make_ndx -f 3HTB_JZ4_min.gro -o 3HTB_JZ4_index.ndx

2022-04-18 23:18:40,871 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 23:18:40,871 [MainThread  ] [INFO ]  Going to read 0 old index file(s)
Analysing residue names:
There are:   163    Protein residues
There are:     1      Other residues
There are:  9505      Water residues
There are:    26        Ion residues
Analysing Protein...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...
Analysing residues not classified as Protein/DNA/RNA/Water and splitting into groups...

  0 System              : 31177 atoms
  1 Protein             :  2614 atoms
  2 Protein-H           :  1301 atoms
  3 C-alpha             :   163 atoms
  4 Backbone   

0

<a id="eqNVTStep2"></a>
### Step 2: Creating portable binary run file for system equilibration (NVT)
Note that for the purposes of temperature coupling, the **protein-ligand complex** (*Protein_Other*) is considered as a single entity.

In [113]:
# Grompp: Creating portable binary run file for NVT System Equilibration
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnvt_tpr = pdbCode+'_'+ligandCode+'gppnvt.tpr'
prop = {
    'mdp':{
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'nvt'
}

# Create and launch bb
grompp(input_gro_path=output_min_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnvt_tpr,
       properties=prop)

2022-04-18 23:20:37,374 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-18 23:20:37,380 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_genion_top.zip
2022-04-18 23:20:37,381 [MainThread  ] [INFO ]  to:
2022-04-18 23:20:37,382 [MainThread  ] [INFO ]  ['5e38f2f1-21b2-4e01-9b43-77bc0385daa4/JZ4_params.itp', '5e38f2f1-21b2-4e01-9b43-77bc0385daa4/JZ4_posres.itp', '5e38f2f1-21b2-4e01-9b43-77bc0385daa4/ligand.top', '5e38f2f1-21b2-4e01-9b43-77bc0385daa4/posre.itp']
2022-04-18 23:20:37,382 [MainThread  ] [INFO ]  Unzipping: 
2022-04-18 23:20:37,383 [MainThread  ] [INFO ]  3HTB_JZ4_genion_top.zip
2022-04-18 23:20:37,384 [MainThread  ] [INFO ]  To: 
2022-04-18 23:20:37,385 [MainThread  ] [INFO ]  5e38f2f1-21b2-4e01-9b43-77bc0385daa4/JZ4_params.itp
2022-04-18 23:20:37,385 [MainThread  ] [INFO ]  5e38f2f1-21b2-4e01-9b43-77bc0385daa4/JZ4_posres.itp
2022-04-18 23:20:37,386 [MainThread  ] [INFO ]  5e38f2f1-21b2-4e01-9b43-77bc0385daa4/li

0

<a id="eqNVTStep3"></a>
### Step 3: Running NVT equilibration

In [115]:
# Mdrun: Running NVT System Equilibration 
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_nvt_trr = pdbCode+'_'+ligandCode+'_nvt.trr'
output_nvt_gro = pdbCode+'_'+ligandCode+'_nvt.gro'
output_nvt_edr = pdbCode+'_'+ligandCode+'_nvt.edr'
output_nvt_log = pdbCode+'_'+ligandCode+'_nvt.log'
output_nvt_cpt = pdbCode+'_'+ligandCode+'_nvt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnvt_tpr,
      output_trr_path=output_nvt_trr,
      output_gro_path=output_nvt_gro,
      output_edr_path=output_nvt_edr,
      output_log_path=output_nvt_log,
      output_cpt_path=output_nvt_cpt)

2022-04-18 23:47:51,121 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-18 23:47:51,126 [MainThread  ] [INFO ]  Not using any container
2022-04-18 23:50:02,033 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4gppnvt.tpr -o 3HTB_JZ4_nvt.trr -c 3HTB_JZ4_nvt.gro -e 3HTB_JZ4_nvt.edr -g 3HTB_JZ4_nvt.log -cpo 3HTB_JZ4_nvt.cpt

2022-04-18 23:50:02,035 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 23:50:02,035 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/biobb_md/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4gppnvt.tpr -o 3HTB_JZ4_nvt.trr -c 3HTB_JZ4_nvt.gro -e 3HTB_JZ4_nvt.edr -g 3HTB_JZ4_nvt.log -cpo 3HTB_JZ4_nvt.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than t

0

<a id="eqNVTStep4"></a>
### Step 4: Checking NVT Equilibration results
Checking **NVT Equilibration** results. Plotting **system temperature** by time during the NVT equilibration process. 

In [116]:
# GMXEnergy: Getting system temperature by time during NVT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_nvt_temp_xvg = pdbCode + '_' + ligandCode + '_nvt_temp.xvg'
prop = {
    'terms':  ["Temperature"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_nvt_edr, 
          output_xvg_path=output_nvt_temp_xvg, 
          properties=prop)

2022-04-18 23:56:54,284 [MainThread  ] [INFO ]  Not using any container
2022-04-18 23:56:54,302 [MainThread  ] [INFO ]  gmx energy -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_nvt.edr -o 3HTB_JZ4_nvt_temp.xvg -xvg none < 18f01f0f-84ef-4f7a-9d49-84e599e4d2e0/instructions.in

2022-04-18 23:56:54,303 [MainThread  ] [INFO ]  Exit code 0

2022-04-18 23:56:54,305 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 1 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Temperature                 298.178        1.8    8.89853     9.6089  (K)

2022-04-18 23:56:54,306 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Bl

0

In [117]:
import plotly
import plotly.graph_objs as go

# Read temperature data from file 
with open(output_nvt_temp_xvg,'r') as temperature_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in temperature_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Temperature during NVT Equilibration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Temperature (K)")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig2.png' width="50%" height="50%"></img>

<a id="npt"></a>
***
## Equilibrate the system (NPT)
Equilibrate the **protein-ligand complex** system in NPT ensemble (constant Number of particles, Pressure and Temperature) .
- [Step 1](#eqNPTStep1): Creating portable binary run file for system equilibration
- [Step 2](#eqNPTStep2): Equilibrate the **protein-ligand complex** with NPT ensemble.
- [Step 3](#eqNPTStep3): Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process.
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXEnergy](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_energy) from **biobb_analysis.gromacs.gmx_energy** 
***

<a id="eqNPTStep1"></a>
### Step 1: Creating portable binary run file for system equilibration (NPT)

In [118]:
# Grompp: Creating portable binary run file for (NPT) System Equilibration
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
output_gppnpt_tpr = pdbCode+'_'+ligandCode+'_gppnpt.tpr'
prop = {
    'mdp':{
        'type': 'npt',
        'nsteps':'5000',
        'tc-grps': 'Protein_Other Water_and_ions',
        'Define': '-DPOSRES -D' + posresifdef
    },
    'simulation_type':'npt'
}

# Create and launch bb
grompp(input_gro_path=output_nvt_gro,
       input_top_zip_path=output_genion_top_zip,
       input_ndx_path=output_complex_ndx,
       output_tpr_path=output_gppnpt_tpr,
       input_cpt_path=output_nvt_cpt,
       properties=prop)

2022-04-19 00:04:22,642 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-19 00:04:22,662 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_genion_top.zip
2022-04-19 00:04:22,663 [MainThread  ] [INFO ]  to:
2022-04-19 00:04:22,664 [MainThread  ] [INFO ]  ['501e33db-c4ed-4c40-9b54-dbaf3d7b3981/JZ4_params.itp', '501e33db-c4ed-4c40-9b54-dbaf3d7b3981/JZ4_posres.itp', '501e33db-c4ed-4c40-9b54-dbaf3d7b3981/ligand.top', '501e33db-c4ed-4c40-9b54-dbaf3d7b3981/posre.itp']
2022-04-19 00:04:22,665 [MainThread  ] [INFO ]  Unzipping: 
2022-04-19 00:04:22,666 [MainThread  ] [INFO ]  3HTB_JZ4_genion_top.zip
2022-04-19 00:04:22,667 [MainThread  ] [INFO ]  To: 
2022-04-19 00:04:22,668 [MainThread  ] [INFO ]  501e33db-c4ed-4c40-9b54-dbaf3d7b3981/JZ4_params.itp
2022-04-19 00:04:22,669 [MainThread  ] [INFO ]  501e33db-c4ed-4c40-9b54-dbaf3d7b3981/JZ4_posres.itp
2022-04-19 00:04:22,670 [MainThread  ] [INFO ]  501e33db-c4ed-4c40-9b54-dbaf3d7b3981/li

0

<a id="eqNPTStep2"></a>
### Step 2: Running NPT equilibration

In [119]:
# Mdrun: Running NPT System Equilibration
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_npt_trr = pdbCode+'_'+ligandCode+'_npt.trr'
output_npt_gro = pdbCode+'_'+ligandCode+'_npt.gro'
output_npt_edr = pdbCode+'_'+ligandCode+'_npt.edr'
output_npt_log = pdbCode+'_'+ligandCode+'_npt.log'
output_npt_cpt = pdbCode+'_'+ligandCode+'_npt.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppnpt_tpr,
      output_trr_path=output_npt_trr,
      output_gro_path=output_npt_gro,
      output_edr_path=output_npt_edr,
      output_log_path=output_npt_log,
      output_cpt_path=output_npt_cpt)

2022-04-19 00:05:54,597 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-19 00:05:54,602 [MainThread  ] [INFO ]  Not using any container
2022-04-19 00:08:17,541 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppnpt.tpr -o 3HTB_JZ4_npt.trr -c 3HTB_JZ4_npt.gro -e 3HTB_JZ4_npt.edr -g 3HTB_JZ4_npt.log -cpo 3HTB_JZ4_npt.cpt

2022-04-19 00:08:17,542 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 00:08:17,543 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/biobb_md/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppnpt.tpr -o 3HTB_JZ4_npt.trr -c 3HTB_JZ4_npt.gro -e 3HTB_JZ4_npt.edr -g 3HTB_JZ4_npt.log -cpo 3HTB_JZ4_npt.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than

0

<a id="eqNPTStep3"></a>
### Step 3: Checking NPT Equilibration results
Checking **NPT Equilibration** results. Plotting **system pressure and density** by time during the **NPT equilibration** process. 

In [120]:
# GMXEnergy: Getting system pressure and density by time during NPT Equilibration  
from biobb_analysis.gromacs.gmx_energy import gmx_energy

# Create prop dict and inputs/outputs
output_npt_pd_xvg = pdbCode+'_'+ligandCode+'_npt_PD.xvg'
prop = {
    'terms':  ["Pressure","Density"]
}

# Create and launch bb
gmx_energy(input_energy_path=output_npt_edr, 
          output_xvg_path=output_npt_pd_xvg, 
          properties=prop)

2022-04-19 00:09:06,154 [MainThread  ] [INFO ]  Not using any container
2022-04-19 00:09:06,273 [MainThread  ] [INFO ]  gmx energy -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_npt.edr -o 3HTB_JZ4_npt_PD.xvg -xvg none < dd5752e2-5820-4d13-bc17-c6802da76711/instructions.in

2022-04-19 00:09:06,276 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 00:09:06,277 [MainThread  ] [INFO ]  
Statistics over 5001 steps [ 0.0000 through 10.0000 ps ], 2 data sets
All statistics are over 51 points

Energy                      Average   Err.Est.       RMSD  Tot-Drift
-------------------------------------------------------------------------------
Pressure                   -2.70674         24    214.153     111.76  (bar)
Density                     1025.71        2.3    6.47818    14.3198  (kg/m^3)

2022-04-19 00:09:06,279 [MainThread  ] [INFO ]                        :-) GROMACS - gmx energy, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostol

0

In [121]:
import plotly
from plotly import subplots
import plotly.graph_objs as go

# Read pressure and density data from file 
with open(output_npt_pd_xvg,'r') as pd_file:
    x,y,z = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]),float(line.split()[2]))
            for line in pd_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

trace1 = go.Scatter(
    x=x,y=y
)
trace2 = go.Scatter(
    x=x,y=z
)

fig = subplots.make_subplots(rows=1, cols=2, print_grid=False)

fig.append_trace(trace1, 1, 1)
fig.append_trace(trace2, 1, 2)

fig['layout']['xaxis1'].update(title='Time (ps)')
fig['layout']['xaxis2'].update(title='Time (ps)')
fig['layout']['yaxis1'].update(title='Pressure (bar)')
fig['layout']['yaxis2'].update(title='Density (Kg*m^-3)')

fig['layout'].update(title='Pressure and Density during NPT Equilibration')
fig['layout'].update(showlegend=False)

plotly.offline.iplot(fig)

<img src='images/fig3.png' width="50%" height="50%"></img>

<a id="free"></a>
***
## Free Molecular Dynamics Simulation
Upon completion of the **two equilibration phases (NVT and NPT)**, the system is now well-equilibrated at the desired temperature and pressure. The **position restraints** can now be released. The last step of the **protein-ligand complex** MD setup is a short, **free MD simulation**, to ensure the robustness of the system. 
- [Step 1](#mdStep1): Creating portable binary run file to run a **free MD simulation**.
- [Step 2](#mdStep2): Run short MD simulation of the **protein-ligand complex**.
- [Step 3](#mdStep3): Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. 
***
**Building Blocks** used:
 - [Grompp](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.grompp) from **biobb_md.gromacs.grompp** 
 - [Mdrun](https://biobb-md.readthedocs.io/en/latest/gromacs.html#module-gromacs.mdrun) from **biobb_md.gromacs.mdrun** 
 - [GMXRms](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rms) from **biobb_analysis.gromacs.gmx_rms**:
     * Wrapper of the GROMACS rms module for performing a Root Mean Square deviation (RMSd) analysis from a given GROMACS compatible trajectory.
     
     
 - [GMXRgyr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_rgyr) from **biobb_analysis.gromacs.gmx_rgyr**:
     * Wrapper of the GROMACS gyrate module for computing the radius of gyration (Rgyr) of a molecule about the x-, y- and z-axes, as a function of time, from a given GROMACS compatible trajectory.
***

<a id="mdStep1"></a>
### Step 1: Creating portable binary run file to run a free MD simulation

In [122]:
# Grompp: Creating portable binary run file for mdrun
from biobb_md.gromacs.grompp import grompp

# Create prop dict and inputs/outputs
prop = {
    'mdp':{
        #'nsteps':'500000' # 1 ns (500,000 steps x 2fs per step)
        #'nsteps':'5000' # 10 ps (5,000 steps x 2fs per step)
        'nsteps':'25000' # 50 ps (25,000 steps x 2fs per step)
    },
    'simulation_type':'free'
}
output_gppmd_tpr = pdbCode+'_'+ligandCode + '_gppmd.tpr'

# Create and launch bb
grompp(input_gro_path=output_npt_gro,
       input_top_zip_path=output_genion_top_zip,
       output_tpr_path=output_gppmd_tpr,
       input_cpt_path=output_npt_cpt,
       properties=prop)

2022-04-19 00:13:06,171 [MainThread  ] [INFO ]  GROMACS Grompp 20191 version detected
2022-04-19 00:13:06,182 [MainThread  ] [INFO ]  Extracting: /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_genion_top.zip
2022-04-19 00:13:06,186 [MainThread  ] [INFO ]  to:
2022-04-19 00:13:06,189 [MainThread  ] [INFO ]  ['d68d6e33-785b-433b-971e-09d7d061e490/JZ4_params.itp', 'd68d6e33-785b-433b-971e-09d7d061e490/JZ4_posres.itp', 'd68d6e33-785b-433b-971e-09d7d061e490/ligand.top', 'd68d6e33-785b-433b-971e-09d7d061e490/posre.itp']
2022-04-19 00:13:06,193 [MainThread  ] [INFO ]  Unzipping: 
2022-04-19 00:13:06,197 [MainThread  ] [INFO ]  3HTB_JZ4_genion_top.zip
2022-04-19 00:13:06,201 [MainThread  ] [INFO ]  To: 
2022-04-19 00:13:06,202 [MainThread  ] [INFO ]  d68d6e33-785b-433b-971e-09d7d061e490/JZ4_params.itp
2022-04-19 00:13:06,204 [MainThread  ] [INFO ]  d68d6e33-785b-433b-971e-09d7d061e490/JZ4_posres.itp
2022-04-19 00:13:06,206 [MainThread  ] [INFO ]  d68d6e33-785b-433b-971e-09d7d061e490/li

0

<a id="mdStep2"></a>
### Step 2: Running short free MD simulation

In [123]:
# Mdrun: Running free dynamics
from biobb_md.gromacs.mdrun import mdrun

# Create prop dict and inputs/outputs
output_md_trr = pdbCode+'_'+ligandCode+'_md.trr'
output_md_gro = pdbCode+'_'+ligandCode+'_md.gro'
output_md_edr = pdbCode+'_'+ligandCode+'_md.edr'
output_md_log = pdbCode+'_'+ligandCode+'_md.log'
output_md_cpt = pdbCode+'_'+ligandCode+'_md.cpt'

# Create and launch bb
mdrun(input_tpr_path=output_gppmd_tpr,
      output_trr_path=output_md_trr,
      output_gro_path=output_md_gro,
      output_edr_path=output_md_edr,
      output_log_path=output_md_log,
      output_cpt_path=output_md_cpt)

2022-04-19 00:14:16,372 [MainThread  ] [INFO ]  GROMACS Mdrun 20191 version detected
2022-04-19 00:14:16,373 [MainThread  ] [INFO ]  Not using any container
2022-04-19 00:38:00,767 [MainThread  ] [INFO ]  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppmd.tpr -o 3HTB_JZ4_md.trr -c 3HTB_JZ4_md.gro -e 3HTB_JZ4_md.edr -g 3HTB_JZ4_md.log -cpo 3HTB_JZ4_md.cpt

2022-04-19 00:38:00,773 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 00:38:00,779 [MainThread  ] [INFO ]                        :-) GROMACS - gmx mdrun, 2019.1 (-:

Executable:   /opt/miniconda3/envs/biobb_env/bin/gmx
Data prefix:  /opt/miniconda3/envs/biobb_env
Working dir:  /home/lg/repos/javerianaMD/biobb_md/notebooks
Command line:
  gmx -nobackup -nocopyright mdrun -s 3HTB_JZ4_gppmd.tpr -o 3HTB_JZ4_md.trr -c 3HTB_JZ4_md.gro -e 3HTB_JZ4_md.edr -g 3HTB_JZ4_md.log -cpo 3HTB_JZ4_md.cpt

Compiled SIMD: SSE2, but for this host/run AVX2_256 might be better (see log).
The current CPU can measure timings more accurately than the code in

0

<a id="mdStep3"></a>
### Step 3: Checking free MD simulation results
Checking results for the final step of the setup process, the **free MD run**. Plotting **Root Mean Square deviation (RMSd)** and **Radius of Gyration (Rgyr)** by time during the **free MD run** step. **RMSd** against the **experimental structure** (input structure of the pipeline) and against the **minimized and equilibrated structure** (output structure of the NPT equilibration step).

In [126]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against minimized and equilibrated snapshot (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_first = pdbCode+'_'+ligandCode+'_rms_first.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms(input_structure_path=output_gppmd_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_first, 
          properties=prop)

2022-04-19 08:58:30,332 [MainThread  ] [INFO ]  Not using any container
2022-04-19 08:58:30,471 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_gppmd.tpr -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_md.trr -o 3HTB_JZ4_rms_first.xvg -xvg none

2022-04-19 08:58:30,472 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 08:58:30,473 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2022-04-19 08:58:30,474 [MainThread  ] [INFO ]                         :-) GROMACS - gmx rms, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jord

0

In [127]:
# GMXRms: Computing Root Mean Square deviation to analyse structural stability 
#         RMSd against experimental structure (backbone atoms)   

from biobb_analysis.gromacs.gmx_rms import gmx_rms

# Create prop dict and inputs/outputs
output_rms_exp = pdbCode+'_'+ligandCode+'_rms_exp.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rms (input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rms_exp, 
         properties=prop)

2022-04-19 08:58:36,839 [MainThread  ] [INFO ]  Not using any container
2022-04-19 08:58:36,976 [MainThread  ] [INFO ]  echo 'Backbone Backbone' | gmx rms -s /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_gppmin.tpr -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_md.trr -o 3HTB_JZ4_rms_exp.xvg -xvg none

2022-04-19 08:58:36,977 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 08:58:36,978 [MainThread  ] [INFO ]  Selected 4: 'Backbone'
Selected 4: 'Backbone'

2022-04-19 08:58:36,978 [MainThread  ] [INFO ]                         :-) GROMACS - gmx rms, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jorda

0

In [128]:
import plotly
import plotly.graph_objs as go

# Read RMS vs first snapshot data from file 
with open(output_rms_first,'r') as rms_first_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_first_file 
            if not line.startswith(("#","@")) 
        ])
    )

# Read RMS vs experimental structure data from file 
with open(output_rms_exp,'r') as rms_exp_file:
    x2,y2 = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rms_exp_file
            if not line.startswith(("#","@")) 
        ])
    )
    
trace1 = go.Scatter(
    x = x,
    y = y,
    name = 'RMSd vs first'
)

trace2 = go.Scatter(
    x = x,
    y = y2,
    name = 'RMSd vs exp'
)

data = [trace1, trace2]

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": data,
    "layout": go.Layout(title="RMSd during free MD Simulation",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "RMSd (nm)")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig4.png' width="50%" height="50%"></img>

In [129]:
# GMXRgyr: Computing Radius of Gyration to measure the protein compactness during the free MD simulation 

from biobb_analysis.gromacs.gmx_rgyr import gmx_rgyr

# Create prop dict and inputs/outputs
output_rgyr = pdbCode+'_'+ligandCode+'_rgyr.xvg'
prop = {
    'selection':  'Backbone'
}

# Create and launch bb
gmx_rgyr(input_structure_path=output_gppmin_tpr,
         input_traj_path=output_md_trr,
         output_xvg_path=output_rgyr, 
         properties=prop)

2022-04-19 09:03:05,041 [MainThread  ] [INFO ]  Not using any container
2022-04-19 09:03:05,151 [MainThread  ] [INFO ]  echo "Backbone" | gmx gyrate -s /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_gppmin.tpr -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_md.trr -o 3HTB_JZ4_rgyr.xvg -xvg none

2022-04-19 09:03:05,153 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 09:03:05,153 [MainThread  ] [INFO ]  Selected 4: 'Backbone'

2022-04-19 09:03:05,155 [MainThread  ] [INFO ]                        :-) GROMACS - gmx gyrate, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenstra       Alan Gray     
  Gerrit Groenhof     Anca Hamuraru    Vincent Hindriksen  M. Eric Irrgang  
  Aleksei Iupinov   Christoph Junghans     Joe Jordan     Dimitrios Karkoulis
    

0

In [130]:
import plotly
import plotly.graph_objs as go

# Read Rgyr data from file 
with open(output_rgyr,'r') as rgyr_file:
    x,y = map(
        list,
        zip(*[
            (float(line.split()[0]),float(line.split()[1]))
            for line in rgyr_file 
            if not line.startswith(("#","@")) 
        ])
    )

plotly.offline.init_notebook_mode(connected=True)

fig = ({
    "data": [go.Scatter(x=x, y=y)],
    "layout": go.Layout(title="Radius of Gyration",
                        xaxis=dict(title = "Time (ps)"),
                        yaxis=dict(title = "Rgyr (nm)")
                       )
})

plotly.offline.iplot(fig)

<img src='images/fig5.png' width="50%" height="50%"></img>

<a id="post"></a>
***
## Post-processing and Visualizing resulting 3D trajectory
Post-processing and Visualizing the **protein-ligand complex system** MD setup **resulting trajectory** using **NGL**
- [Step 1](#ppStep1): *Imaging* the resulting trajectory, **stripping out water molecules and ions** and **correcting periodicity issues**.
- [Step 2](#ppStep2): Generating a *dry* structure, **removing water molecules and ions** from the final snapshot of the MD setup pipeline.
- [Step 3](#ppStep3): Visualizing the *imaged* trajectory using the *dry* structure as a **topology**. 
***
**Building Blocks** used:
 - [GMXImage](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_image) from **biobb_analysis.gromacs.gmx_image**:
     * Wrapper of the GROMACS trjconv module for correcting periodicity (image) from a given GROMACS compatible trajectory file.
     
     
 - [GMXTrjConvStr](https://biobb-analysis.readthedocs.io/en/latest/gromacs.html#module-gromacs.gmx_trjconv_str) from **biobb_analysis.gromacs.gmx_trjconv_str**:
     * Wrapper of the GROMACS trjconv module for converting between GROMACS compatible structure file formats and/or extracting a selection of atoms.
***

<a id="ppStep1"></a>
### Step 1: *Imaging* the resulting trajectory.
Stripping out **water molecules and ions** and **correcting periodicity issues**  

In [131]:
# GMXImage: "Imaging" the resulting trajectory
#           Removing water molecules and ions from the resulting structure
from biobb_analysis.gromacs.gmx_image import gmx_image

# Create prop dict and inputs/outputs
output_imaged_traj = pdbCode+'_imaged_traj.trr'
prop = {
    'center_selection':  'Protein_Other',
    'output_selection': 'Protein_Other',
    'pbc' : 'mol',
    'center' : True
}

# Create and launch bb
gmx_image (input_traj_path  = output_md_trr,
           input_top_path   = output_gppmd_tpr,
           input_index_path = output_complex_ndx,
           output_traj_path = output_imaged_traj, 
           properties       = prop)

2022-04-19 09:10:08,755 [MainThread  ] [INFO ]  Not using any container
2022-04-19 09:10:08,895 [MainThread  ] [INFO ]  echo "Protein_Other" "Protein_Other" | gmx trjconv -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_md.trr -s /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_gppmd.tpr -fit none -o 3HTB_imaged_traj.trr -n /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_index.ndx -center -pbc mol -ur compact

2022-04-19 09:10:08,896 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 09:10:08,897 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for centering
Selected 24: 'Protein_Other'
Select group for output
Selected 24: 'Protein_Other'

2022-04-19 09:10:08,897 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkm

0

<a id="ppStep2"></a>
### Step 2: Generating the output *dry* structure.
**Removing water molecules and ions** from the resulting structure

In [132]:
# GMXTrjConvStr: Converting and/or manipulating a structure
#                Removing water molecules and ions from the resulting structure
#                The "dry" structure will be used as a topology to visualize 
#                the "imaged dry" trajectory generated in the previous step.
from biobb_analysis.gromacs.gmx_trjconv_str import gmx_trjconv_str

# Create prop dict and inputs/outputs
output_dry_gro = pdbCode+'_md_dry.gro'
prop = {
    'selection':  'Protein_Other'
}

# Create and launch bb
gmx_trjconv_str(input_structure_path = output_md_gro,
                input_top_path       = output_gppmd_tpr,
                input_index_path     = output_complex_ndx,
                output_str_path      = output_dry_gro, 
                properties           = prop)

2022-04-19 09:18:12,134 [MainThread  ] [INFO ]  Not using any container
2022-04-19 09:18:12,281 [MainThread  ] [INFO ]  echo "Protein_Other" | gmx trjconv -f /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_md.gro -s /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_gppmd.tpr -o 3HTB_md_dry.gro -n /home/lg/repos/javerianaMD/biobb_md/notebooks/3HTB_JZ4_index.ndx

2022-04-19 09:18:12,283 [MainThread  ] [INFO ]  Exit code 0

2022-04-19 09:18:12,283 [MainThread  ] [INFO ]  Note that major changes are planned in future for trjconv, to improve usability and utility.Select group for output
Selected 24: 'Protein_Other'

2022-04-19 09:18:12,284 [MainThread  ] [INFO ]                       :-) GROMACS - gmx trjconv, 2019.1 (-:

                            GROMACS is written by:
     Emile Apol      Rossen Apostolov      Paul Bauer     Herman J.C. Berendsen
    Par Bjelkmar      Christian Blau   Viacheslav Bolnykh     Kevin Boyd    
 Aldert van Buuren   Rudi van Drunen     Anton Feenst

0

<a id="ppStep3"></a>
### Step 3: Visualizing the generated dehydrated trajectory.
Using the **imaged trajectory** (output of the [Post-processing step 1](#ppStep1)) with the **dry structure** (output of the [Post-processing step 2](#ppStep2)) as a topology.

In [134]:
# Show trajectory
view = nglview.show_simpletraj(nglview.SimpletrajTrajectory(output_imaged_traj, output_dry_gro), gui=True)
view

NGLWidget(max_frame=5)

<img src='images/trajectory.gif'></img>

<a id="output"></a>
## Output files

Important **Output files** generated:
 - {{output_md_gro}}: **Final structure** (snapshot) of the MD setup protocol.
 - {{output_md_trr}}: **Final trajectory** of the MD setup protocol.
 - {{output_md_cpt}}: **Final checkpoint file**, with information about the state of the simulation. It can be used to **restart** or **continue** a MD simulation.
 - {{output_gppmd_tpr}}: **Final tpr file**, GROMACS portable binary run input file. This file contains the starting structure of the **MD setup free MD simulation step**, together with the molecular topology and all the simulation parameters. It can be used to **extend** the simulation.
 - {{output_genion_top_zip}}: **Final topology** of the MD system. It is a compressed zip file including a **topology file** (.top) and a set of auxiliar **include topology** files (.itp).

**Analysis** (MD setup check) output files generated:
 - {{output_rms_first}}: **Root Mean Square deviation (RMSd)** against **minimized and equilibrated structure** of the final **free MD run step**.
 - {{output_rms_exp}}: **Root Mean Square deviation (RMSd)** against **experimental structure** of the final **free MD run step**.
 - {{output_rgyr}}: **Radius of Gyration** of the final **free MD run step** of the **setup pipeline**.
 

***
<a id="questions"></a>

## Questions & Comments

Questions, issues, suggestions and comments are really welcome!

* GitHub issues:
    * [https://github.com/bioexcel/biobb](https://github.com/bioexcel/biobb)

* BioExcel forum:
    * [https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library](https://ask.bioexcel.eu/c/BioExcel-Building-Blocks-library)
